In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn
import scipy as sp
import nltk
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import math 
import matplotlib.pyplot as plt
pd.options.display.max_colwidth = 200
%matplotlib inline
#import contractions
from bs4 import BeautifulSoup
import unicodedata
import nltk 
nltk.download('stopwords')
from numpy.random import seed
from numpy.random import randn
from statsmodels.graphics.gofplots import qqplot
#from matplotlib import pyplot
import pylab
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import norm 
from scipy import stats 
from statsmodels.stats.weightstats import ztest
from matplotlib import rcParams

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nazanin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv(r'C:\Users\Nazanin\Downloads\drugsCom_raw\drugsComTest_raw.csv', sep="\t")
df1 = pd.read_csv(r'C:\Users\Nazanin\Downloads\drugsCom_raw\drugsComTrain_raw.csv', sep="\t")
#display(df, df1)

In [3]:
df2 = df.append(df1)
list(df2)

['Unnamed: 0',
 'drugName',
 'condition',
 'review',
 'rating',
 'date',
 'usefulCount']

In [4]:
df2 = df2.reset_index(drop=True)

In [5]:
print(df2.head())

   Unnamed: 0         drugName                     condition  \
0      163740      Mirtazapine                    Depression   
1      206473       Mesalamine  Crohn's Disease, Maintenance   
2      159672          Bactrim       Urinary Tract Infection   
3       39293         Contrave                   Weight Loss   
4       97768  Cyclafem 1 / 35                 Birth Control   

                                                                                                                                                                                                    review  \
0  "I&#039;ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia &amp; anxiety. My doctor suggested and changed ...   
1  "My son has Crohn&#039;s disease and has done very well on the Asacol.  He has no complaints and shows no side effects.  He has taken as many as nine tablets per day at one time.  I&#039;ve been v... 

In [7]:
df2.drugName.value_counts()[:10]

Levonorgestrel                        4930
Etonogestrel                          4421
Ethinyl estradiol / norethindrone     3753
Nexplanon                             2892
Ethinyl estradiol / norgestimate      2790
Ethinyl estradiol / levonorgestrel    2503
Phentermine                           2085
Sertraline                            1868
Escitalopram                          1747
Mirena                                1673
Name: drugName, dtype: int64

In [8]:
df2.condition.value_counts()[:10]

Birth Control      38436
Depression         12164
Pain                8245
Anxiety             7812
Acne                7435
Bipolar Disorde     5604
Insomnia            4904
Weight Loss         4857
Obesity             4757
ADHD                4509
Name: condition, dtype: int64

In [6]:
print(df2[df2['condition'] == 'Birth Control'].review.head())

4     "I have been on this birth control for one cycle. After reading some of the reviews on this type and similar birth controls I was a bit apprehensive to start. Im giving this birth control a 9 out ...
6     "I&#039;ve had the copper coil for about 3 months now. I was really excited at the thought of not taking hormones. I&#039;m good with pain however I nearly fainted with insertion, couldn&#039;t be...
9     "I was on this pill for almost two years. It does work as far as not getting pregnant however my experience at first was it didn&#039;t make a huge difference then 6 or 7 months into it my sex dri...
30    "I absolutely love this product and recommend to everyone. I know everyone&#039;s body is different, so it is not for everyone, but it is not the medicines fault. I have NO negative symptoms since...
37    "I was on this for 5 years (and birth control pills for about 12 years), and would have told you how fabulous it was.  &lt;List all the benefits everyone else has listed,

In [7]:
print(df2[df2['condition'] == 'Anxiety'].review.head())

48                                                                                                                                                                                              "Works for me"
63     "I was prescribed this for onset of anxiety and possible hormonal mood swings. I was not told by my doctor how it would make me feel or how hard coming off of it would be. I took one 37.5 mg capsu...
83     "I did not like this medication. For anxiety, I have also tried Hydroxyzine (Atarax). I guess this is just my personal body chemistry but I actually prefer Atarax to this unlike most people. This ...
133    "I&#039;m a 32 year old male and I&#039;ve been taking buspar for about 10 months. At first it did nothing but make my anxiety worse. I would wake up with to full blown panic attacks and have the ...
208    "Klonopin is a very effective medicine for people such as myself that suffer from debilitating panic disorder and/or PTSD.  This medicine saved me from becoming inst

## NLP 
+ On the Machine Learning phase, we start looking at the text documents of the review column. In order to do that, we need to do some text pre-processing. We process and tokenize corpus of reviews to build features for predictive models. Also, train machine learning models to predict drug rating based on reviews and relevant metadata.


### Text pre-processing

+ Removing tags: Removing unnecessary HTML tags, etc. which do not add much value when analyzing text (The BeautifulSoup library does an excellent job in providing necessary functions for this.).

+ Removing accented characters: Removing  accented characters\letters in the text corpus and convert these characters and standardized into ASCII characters, an example is to convert é to e. 

+ Expanding contractions: Converting contraction to its expanded, original form often helps with text standardization, example of which would be, "do not" to "don’t" and "I would" to "I’d".

+ Removing special characters: Removing special characters and symbols often add to the extra noise in unstructured text. More than often, simple regular expressions (regexes) can be used to achieve this.

+ Stemming and lemmatization: The reverse process of inflection is called stemming, which is basically obtaining the base form of a word. Lemmatization is very similar to stemming, the difference being that the root word from lemmatization is always a lexicographically correct word but the root stem may not be so.

+ Removing stopwords: Words with little or no significance in text corpus are known as stopwords, example of which are:  a, an, the. We can use a standard English language stopwords list from nltk and also add our own domain specific stopwords as needed.

+ Other text cleaning to do would be tokenization, removing extra whitespaces, text lower casing as well as spelling corrections, grammatical error corrections, removing repeated characters to name a few. 

In [ ]:
#en_stopws = stopwords.words('english')  # this loads the default stopwords list for English
#en_stopws.append('</span> users found this comment helpful.')  # add any words you don't like to the list </span> users found this comment helpful.

In [5]:
# Let's remove special characters, extra whitespaces, digits, stopwords and lower casing the text corpus
#result = re.sub(pattern, repl, string, count=0, flags=0);
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)   ###plscheck this 
wpt = nltk.WordPunctTokenizer()  # $4.99 
stop_words = nltk.corpus.stopwords.words('english')
stop_words.append('</span> users found this comment helpful.') 
stop_words.append('&#039')
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A) 
    #re.I A means treats . as whatever it is. Performs case-insensitive matching. #issues with FLAGS!!!!!
    doc = doc.lower()
    doc = doc.strip()
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words] 
    filtered_tokens = [stemmer.stem(word) for word in filtered_tokens]  
    doc = ' '.join(filtered_tokens)
    return doc
normalize_corpus = np.vectorize(normalize_document)
# doc is the list of strings

In [6]:
norm_corpus = normalize_corpus(df2[df2['condition'] == 'Birth Control'].review)

In [7]:
norm_corpus

array(['birth control one cycl read review type similar birth control bit apprehens start im give birth control long enough far love birth control side effect minim like im even birth control experienc mild headach nausea ive feel great got period cue third day inact pill idea come zero pms period light bare cramp unprotect sex first month obvious didnt get pregnant im pleas high recommend',
       'ive copper coil month realli excit thought take hormon im good pain howev near faint insert couldnt beliv pain doctor say pain well month period last day im pain day random twang especi left side im consid whether want put intens pain heavi period id recommend somebodi doesnt alreadi heavi pain period right isnt',
       'pill almost two year work far get pregnant howev experi first didnt make huge differ month sex drive went along dri moodi increas drastic would cri one second get angri husband anyth everyth skin gotten lot wors broke place never last week yaz',
       ...,
       'experi 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [8]:
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [9]:
get_top_n_words(normalize_corpus(df2[df2['condition'] == 'Birth Control'].review), n=40)  #None

[('period', 37273),
 ('month', 36515),
 ('pill', 26390),
 ('get', 23071),
 ('ive', 21923),
 ('day', 21659),
 ('im', 20430),
 ('year', 19498),
 ('control', 18934),
 ('week', 18639),
 ('birth', 18338),
 ('take', 18146),
 ('first', 17184),
 ('cramp', 16278),
 ('gain', 15381),
 ('weight', 15367),
 ('start', 15324),
 ('effect', 14001),
 ('got', 13875),
 ('time', 13647),
 ('pain', 13167),
 ('like', 13144),
 ('side', 12075),
 ('bleed', 11932),
 ('would', 11297),
 ('insert', 11113),
 ('feel', 10858),
 ('sinc', 10519),
 ('mood', 10392),
 ('acn', 10089),
 ('never', 9849),
 ('spot', 9770),
 ('sex', 9433),
 ('bad', 9376),
 ('go', 9145),
 ('use', 9100),
 ('also', 9021),
 ('one', 8951),
 ('last', 8476),
 ('dont', 8321)]

In [10]:
vectorizer = CountVectorizer(ngram_range=(2,2), min_df=0.05, max_df=1.0, max_features=500)

# call `fit` to build the vocabulary
vectorizer.fit(norm_corpus)

# call `transform` to convert text to a bag of words
x = vectorizer.transform(norm_corpus)

# CountVectorizer uses a sparse array to save memory, but it's easier in this assignment to 
# convert back to a "normal" numpy array
x = x.toarray()  # this is for visualization purposes 

print()
print("Transformed text vector is \n", x)

# `get_feature_names` tracks which word is associated with each column of the transformed x
print
print("Words for each feature:")
print(vectorizer.get_feature_names())  # visualization 


Transformed text vector is 
 [[5 0 1 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 1]
 [2 0 0 ... 0 0 0]]
Words for each feature:
['birth control', 'feel like', 'first month', 'gain weight', 'mood swing', 'sex drive', 'side effect', 'take pill', 'weight gain']


In [11]:
x.shape

(38436, 9)

In [12]:
y = df2[df2['condition'] == 'Birth Control'].rating  # df2['rating']
xtrain, xtest, ytrain, ytest = train_test_split(x, y)  #(default=0.25)
clf = MultinomialNB().fit(xtrain, ytrain)

In [13]:
training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

In [14]:
print("MN Accuracy: %0.2f%%" % (100 * clf.score(xtest, ytest)))
print("Accuracy on training data: %0.2f" % (training_accuracy))
print("Accuracy on test data:     %0.2f" % (test_accuracy))   #30% is precicely predicting the ratings for df2_min = 0.1
# do a good and bad as a binary! 

MN Accuracy: 20.79%
Accuracy on training data: 0.21
Accuracy on test data:     0.21


In [19]:
 #regressor target variables rating is 
# Train the model with Random Foprest Classifier     
clf_RF = RandomForestClassifier()  
#n_jobs=2 number of cores the computer uses, n_estimators = 500 (number of trees)  startt 50 
clf_RF.fit(xtrain, ytrain)  
# predict and evaluate performance
clf_RF_predictions = clf_RF.predict(xtest)
#meu.display_model_performance_metrics(true_labels=ytest, predicted_labels=clf_RF_predictions , classes= range(1,11))
#hash maps a value to another value and it's more efficient in computation
# normaization removing bias that could exists in a data set 
# correcting for the size of the corpus. one word review and the other is 200000 words 
# and contains the word good and we should 
# normaliztion in NLP is to use word frequecy. 

C:\Users\Nazanin\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [20]:
print(clf_RF.feature_importances_)

[0.20584546 0.06917504 0.0826262  0.08909746 0.15301874 0.08619107
 0.10124585 0.12122744 0.09157273]


In [23]:
print(clf_RF.get_params())

{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [25]:
#gridsearch and hyoerparameter tuning (on the TRAIN set) 
param_grid = {'n_estimators': [100, 200, 300, 500], 'max_features': ['auto', None, 'log2']}
clf_RF = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
clf_RF.fit(xtrain, ytrain)
print(clf_RF.best_params_)

{'max_features': None, 'n_estimators': 100}


In [29]:
results_22gram = clf_RF.cv_results_
for param, score_mean, score_sd in zip(results['params'], results['mean_test_score'], results['std_test_score']):
    print(param, round(score_mean, 4), round(score_sd, 4))

{'max_features': 'auto', 'n_estimators': 100} 0.2268 0.0033
{'max_features': 'auto', 'n_estimators': 200} 0.2258 0.003
{'max_features': 'auto', 'n_estimators': 300} 0.2267 0.0026
{'max_features': 'auto', 'n_estimators': 500} 0.2268 0.003
{'max_features': None, 'n_estimators': 100} 0.2269 0.0036
{'max_features': None, 'n_estimators': 200} 0.2259 0.0028
{'max_features': None, 'n_estimators': 300} 0.2266 0.0027
{'max_features': None, 'n_estimators': 500} 0.2268 0.0028
{'max_features': 'log2', 'n_estimators': 100} 0.2268 0.0033
{'max_features': 'log2', 'n_estimators': 200} 0.2258 0.003
{'max_features': 'log2', 'n_estimators': 300} 0.2267 0.0026
{'max_features': 'log2', 'n_estimators': 500} 0.2268 0.003


In [30]:
# The best hyperparameter performance occurs at: {'max_features': None, 'n_estimators': 100} 0.2269 0.0036

In [31]:
clf_RF = RandomForestClassifier(n_estimators=100, max_features=None, random_state=42)
clf_RF.fit(xtrain, ytrain)
clf_RF_predictions = clf_RF.predict(xtest)

In [32]:
print(confusion_matrix(ytest,clf_RF_predictions))  
print(classification_report(ytest,clf_RF_predictions))  
print(accuracy_score(ytest, clf_RF_predictions)) 

[[ 296   20   16    7    9    6   12   36   70  990]
 [ 145   23    7    6    8    4   11   30   45  464]
 [ 108   21   38    3    6    3    6   27   44  426]
 [ 100   11    3   14   11    6   12   29   30  297]
 [ 128   17    5    7   26    4    4   38   55  468]
 [  73   17    4    4    4   27    7   22   43  280]
 [  79    7    6    6    9    2   24   33   61  385]
 [ 144   17    6    7   14    6   17  123   63  651]
 [ 159   14   20    3   10    6   15   66  194  937]
 [ 169   29   10    3   10    7   17   75  125 1447]]
              precision    recall  f1-score   support

         1.0       0.21      0.20      0.21      1462
         2.0       0.13      0.03      0.05       743
         3.0       0.33      0.06      0.10       682
         4.0       0.23      0.03      0.05       513
         5.0       0.24      0.03      0.06       752
         6.0       0.38      0.06      0.10       481
         7.0       0.19      0.04      0.07       612
         8.0       0.26      0.12   

In [33]:
from sklearn.datasets import make_classification
clf_RF.score(x, y, sample_weight=None)  # on the entire data here so obviously it goes up by a few percentages 

0.2560099906337808

Now, we are performing ngram(1,2): 

In [34]:
# So now, let's change the ngram to (1,2)
vectorizer = CountVectorizer(ngram_range=(1,2), min_df=0.05, max_df=1.0, max_features=500)

# call `fit` to build the vocabulary
vectorizer.fit(norm_corpus)

# call `transform` to convert text to a bag of words
x = vectorizer.transform(norm_corpus)

# CountVectorizer uses a sparse array to save memory, but it's easier in this assignment to 
# convert back to a "normal" numpy array
x = x.toarray()  # this is for visualization purposes 

print()
print("Transformed text vector is \n", x)

# `get_feature_names` tracks which word is associated with each column of the transformed x
print
print("Words for each feature:")
print(vectorizer.get_feature_names())  # visualization 


Transformed text vector is 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 1]]
Words for each feature:
['absolut', 'acn', 'actual', 'ago', 'almost', 'also', 'alway', 'anoth', 'anxieti', 'anyth', 'around', 'away', 'back', 'bad', 'bc', 'best', 'better', 'birth', 'birth control', 'bit', 'bleed', 'bloat', 'bodi', 'break', 'breast', 'cant', 'caus', 'chang', 'clear', 'come', 'complet', 'constant', 'control', 'could', 'coupl', 'cramp', 'crazi', 'cri', 'day', 'decid', 'definit', 'depress', 'didnt', 'differ', 'doctor', 'dont', 'drive', 'due', 'eat', 'effect', 'emot', 'end', 'even', 'ever', 'everi', 'everyon', 'everyth', 'experi', 'experienc', 'extrem', 'face', 'far', 'feel', 'feel like', 'felt', 'fine', 'first', 'first month', 'gain', 'gain weight', 'get', 'give', 'go', 'good', 'got', 'gotten', 'great', 'half', 'happi', 'havent', 'headach', 'heavi', 'help', 'high', 'hope', 'hormon', 'horribl', 'hour', 'howev', 'hurt', 'im', 

In [35]:
x.shape

(38436, 199)

In [36]:
y = df2[df2['condition'] == 'Birth Control'].rating  # df2['rating']
xtrain, xtest, ytrain, ytest = train_test_split(x, y)  #(default=0.25)

In [37]:
 #regressor target variables rating is 
# Train the model with Random Foprest Classifier     
clf_RF = RandomForestClassifier()  
#n_jobs=2 number of cores the computer uses, n_estimators = 500 (number of trees)  startt 50 
clf_RF.fit(xtrain, ytrain)  
# predict and evaluate performance
clf_RF_predictions = clf_RF.predict(xtest)

C:\Users\Nazanin\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [38]:
print(clf_RF.feature_importances_)

[0.00261367 0.00780871 0.00252133 0.00427247 0.00475976 0.00751505
 0.00422981 0.0029895  0.00387865 0.00303289 0.003107   0.00344073
 0.00636469 0.00757787 0.00281829 0.00457    0.00369148 0.00854427
 0.00801773 0.00296896 0.00851889 0.00289245 0.00459808 0.0031987
 0.00480537 0.00408266 0.00405533 0.00489358 0.00357263 0.00328636
 0.00311246 0.00455431 0.0087329  0.00345672 0.00323442 0.00871686
 0.00280028 0.00386266 0.01123768 0.00280397 0.00291506 0.00748976
 0.00576805 0.00413464 0.00574328 0.00648376 0.0046758  0.00281816
 0.00275476 0.00655606 0.00385661 0.00273766 0.00479863 0.00475689
 0.00606255 0.00273185 0.00266058 0.00506077 0.00473704 0.00443979
 0.0030949  0.00570814 0.00755933 0.00275802 0.00418533 0.00295539
 0.00919187 0.00404637 0.00882739 0.00310749 0.0119212  0.00311335
 0.00710692 0.00543622 0.00855409 0.00340439 0.00508978 0.00253946
 0.00307303 0.00487165 0.00479056 0.0044822  0.00452421 0.00231088
 0.00449298 0.00381928 0.00534128 0.00272746 0.00457527 0.00290

In [39]:
print(clf_RF.get_params())

{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [40]:
#gridsearch and hyoerparameter tuning (on the TRAIN set) #THIS CELL IS 24K GOLD! 
param_grid = {'n_estimators': [100, 200, 300, 500], 'max_features': ['auto', None, 'log2']}
clf_RF = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
clf_RF.fit(xtrain, ytrain)
print(clf_RF.best_params_)

{'max_features': 'auto', 'n_estimators': 500}


In [42]:
results_12gram = clf_RF.cv_results_
for param, score_mean, score_sd in zip(results['params'], results['mean_test_score'], results['std_test_score']):
    print(param, round(score_mean, 4), round(score_sd, 4))

{'max_features': 'auto', 'n_estimators': 100} 0.2268 0.0033
{'max_features': 'auto', 'n_estimators': 200} 0.2258 0.003
{'max_features': 'auto', 'n_estimators': 300} 0.2267 0.0026
{'max_features': 'auto', 'n_estimators': 500} 0.2268 0.003
{'max_features': None, 'n_estimators': 100} 0.2269 0.0036
{'max_features': None, 'n_estimators': 200} 0.2259 0.0028
{'max_features': None, 'n_estimators': 300} 0.2266 0.0027
{'max_features': None, 'n_estimators': 500} 0.2268 0.0028
{'max_features': 'log2', 'n_estimators': 100} 0.2268 0.0033
{'max_features': 'log2', 'n_estimators': 200} 0.2258 0.003
{'max_features': 'log2', 'n_estimators': 300} 0.2267 0.0026
{'max_features': 'log2', 'n_estimators': 500} 0.2268 0.003


In [43]:
clf_RF = RandomForestClassifier(n_estimators=500, max_features='auto', random_state=42)
clf_RF.fit(xtrain, ytrain)
clf_RF_predictions = clf_RF.predict(xtest)

In [44]:
print(confusion_matrix(ytest,clf_RF_predictions))  
print(classification_report(ytest,clf_RF_predictions))  
print(accuracy_score(ytest, clf_RF_predictions)) 

[[1397    0    2    0    0    0    0    2   15   54]
 [ 140  512    3    0    0    0    0    2    3   37]
 [ 129    0  492    0    6    0    0    2   14   38]
 [  70    2    2  429    2    0    0    4   17   20]
 [ 100    0    4    0  532    0    0   15   31   48]
 [  40    0    0    2    2  360    0   16   33   41]
 [  44    2    0    0    6    0  448   17   52   65]
 [  34    0    2    0    0    0    0  794   72  140]
 [  35    0    0    0    2    0    2   17 1091  212]
 [  36    0    0    0    0    0    0    9   52 1859]]
              precision    recall  f1-score   support

         1.0       0.69      0.95      0.80      1470
         2.0       0.99      0.73      0.84       697
         3.0       0.97      0.72      0.83       681
         4.0       1.00      0.79      0.88       546
         5.0       0.97      0.73      0.83       730
         6.0       1.00      0.73      0.84       494
         7.0       1.00      0.71      0.83       634
         8.0       0.90      0.76   

In [ ]:
from sklearn.datasets import make_classification
clf_RF.score(x, y, sample_weight=None) 

In [45]:
a = pd.DataFrame(data={'imp':clf_RF.feature_importances_,'vars':vectorizer.get_feature_names()})
a

,imp,vars
0,0.002505,absolut
1,0.007727,acn
2,0.002709,actual
3,0.004506,ago
4,0.005318,almost
5,0.007333,also
6,0.004046,alway
7,0.002746,anoth
8,0.003640,anxieti
9,0.002858,anyth


In [46]:
a.sort_values(by = 'imp' , ascending = False)

,imp,vars
113,0.014883,month
130,0.014685,period
108,0.013692,love
131,0.012514,pill
97,0.011939,ive
70,0.011831,get
90,0.011769,im
38,0.011203,day
187,0.010755,week
198,0.010417,year


In [47]:
normalize_corpus_outside = normalize_corpus("i loved this birth control")  # pre process, I do the count vectorizer, model.predict (on the verized form)

In [49]:
vectorizer = CountVectorizer(ngram_range=(1,2), min_df=0.05, max_df=1.0, max_features=500)

In [50]:
vectorizer.fit(normalize_corpus_outside)

TypeError: iteration over a 0-d array

In [51]:
x = vectorizer.transform(norm_corpus)

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

Train, Predict & Evaluate Extreme Gradient Boosted Model with tuned hyperparameters